# ML_MariaJoseCamacho - Predicción de Rotación de Empleados (Employee Attrition)
## **Paso 1: Entendiendo el problema**


Este proyecto busca predecir qué empleados podrían abandonar la empresa, utilizando datos históricos. Anticipar la rotación laboral permite tomar decisiones estratégicas en RRHH.

## **Paso 2: Carga de librerías y datos**


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import joblib

In [4]:
# Cargar los datos
file_path = r"C:\Users\marij\Documents\dataset ML\WA_Fn-UseC_-HR-Employee-Attrition.csv"
df = pd.read_csv(file_path)
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


## **Paso 3: Preparación del dataset**
(Selección de variables, tratamiento de nulos, escalado si es necesario)


In [12]:
# Tamaño fijo para redimensionar las imágenes
IMG_SIZE = (128, 128)

imagenes_aplanadas = []

for img_name in images:
    img_path = os.path.join(DATASET_DIR, img_name)
    # Leer imagen en escala de grises con OpenCV
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    # Redimensionar
    img_resized = cv2.resize(img, IMG_SIZE)
    # Aplanar
    img_flat = img_resized.flatten()
    imagenes_aplanadas.append(img_flat)

# Crear DataFrame
df = pd.DataFrame(imagenes_aplanadas)

# Reordenar aleatoriamente para evitar sesgos
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"DataFrame con {df.shape[0]} imágenes y {df.shape[1]} features (pixeles aplanados)")
df.head()

DataFrame con 670 imágenes y 16384 features (pixeles aplanados)


,0,1,2,3,4,5,6,7,8,9,...,16374,16375,16376,16377,16378,16379,16380,16381,16382,16383
0,229,229,229,229,229,229,229,229,229,229,...,233,233,235,234,233,235,233,236,234,235
1,201,201,201,201,201,201,201,201,201,201,...,158,155,159,161,154,151,159,154,143,150
2,247,247,247,247,247,247,247,247,247,247,...,255,255,255,255,255,255,255,255,255,255
3,245,245,245,245,245,245,245,245,245,245,...,180,185,183,187,189,192,187,182,185,191
4,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255


In [5]:
def extract_features(image_path):
    """
    Función para extraer características simples de una imagen.
    
    - Convierte la imagen a escala de grises.
    - La redimensiona a 128x128 para homogeneizar tamaño.
    - Calcula un histograma normalizado de intensidades (32 bins).
    - Calcula estadísticas básicas: media, desviación estándar, sesgo (skewness), curtosis.
    - Devuelve un vector con todas estas características concatenadas.
    """
    # Abrir imagen y convertir a escala de grises
    img = Image.open(image_path).convert('L')
    
    # Redimensionar imagen a tamaño uniforme para comparabilidad
    img = img.resize((128, 128))
    
    # Convertir imagen a array plano (vector)
    arr = np.array(img).flatten()
    
    # Histograma normalizado de 32 bins entre 0 y 255 (niveles de gris)
    hist, _ = np.histogram(arr, bins=32, range=(0, 255), density=True)
    
    # Estadísticas básicas del vector de pixeles
    mean = np.mean(arr)         # media
    std = np.std(arr)           # desviación estándar
    skewness = skew(arr)        # asimetría
    kurt = kurtosis(arr)        # curtosis
    
    # Concatenamos todas las características en un único vector
    features = np.concatenate((hist, [mean, std, skewness, kurt]))
    return features

# Extraemos características de todas las imágenes del dataset
features = []
for img_name in images:
    img_path = os.path.join(DATASET_DIR, img_name)
    feats = extract_features(img_path)
    features.append(feats)

features = np.array(features)
print(f"Vector de características generado con forma: {features.shape}")

Vector de características generado con forma: (670, 36)


## **Paso 4: Análisis exploratorio (Mini EDA)**
(Distribuciones, correlaciones, visualizaciones de features clave)



## **Paso 5: Reducción de dimensiones (opcional)**
(PCA si decides usarlo para visualización o clustering)

## **Paso 6: Clustering**
(Entrenamiento del modelo: k-means, número óptimo de clusters, silhouette score…)

## **Paso 7: Interpretación de los grupos**
(Describir los perfiles encontrados, visualizarlos en 2D si usaste PCA o t-SNE)

## **Paso 8: Guardado del modelo**
(Exportar modelo entrenado en formato `.joblib` en `/src/models`)

## **Paso 9: Conclusiones**
(Resumen del proyecto y aplicaciones reales del análisis)